## mvcnn
### 一些基础知识
#### mean 均值  std 方差
- 本数据集:
    - means : [0.81662318, 0.81662318, 0.81662318]
    - std: [0.24117189, 0.24117189, 0.24117189]

### 其他
- 需要计算方差

In [2]:
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
import os,shutil,json
import argparse

from tools.Trainer import ModelNetTrainer
from tools.ImgDataset import MultiviewImgDataset, SingleImgDataset
from models.MVCNN import MVCNN, SVCNN

In [3]:
def create_folder(log_dir):
    # make summary folder
    if not os.path.exists(log_dir):
        os.mkdir(log_dir)
    else:
        print('WARNING: summary folder already exists!! It will be overwritten!!')
        shutil.rmtree(log_dir)
        os.mkdir(log_dir)

In [4]:
name = 'mvcnn'
# num_models = 1000
num_models = 0
weight_decay = 0.001 
num_views = 12 
cnn_name = 'vgg11'
no_pretraining = False
lr = 5e-5
bs = 64
batchSize = bs
svcnn_bs = 64
# web_shapent55_renamed_0.2   modelnet40_images_new_12x
train_path = 'web_shapent55_renamed_0.2/*/train'
val_path = 'web_shapent55_renamed_0.2/*/test'

pretraining = not no_pretraining
log_dir = name
create_folder(name)
# config_f = open(os.path.join(log_dir, 'config.json'), 'w')
# json.dump(vars(args), config_f)
# config_f.close()

In [5]:
# test
n_models_train = num_models*num_views

train_dataset = SingleImgDataset(train_path, scale_aug=False, rot_aug=False, num_models=n_models_train, num_views=num_views)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=svcnn_bs, shuffle=True, num_workers=0)

val_dataset = SingleImgDataset(val_path, scale_aug=False, rot_aug=False, test_mode=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=svcnn_bs, shuffle=False, num_workers=0)
print('num_train_files: '+str(len(train_dataset.filepaths)))
print('num_val_files: '+str(len(val_dataset.filepaths)))

train_dataset = MultiviewImgDataset(train_path, scale_aug=False, rot_aug=False, num_models=n_models_train, num_views=num_views)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batchSize, shuffle=False, num_workers=0)# shuffle needs to be false! it's done within the trainer

val_dataset = MultiviewImgDataset(val_path, scale_aug=False, rot_aug=False, num_views=num_views)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batchSize, shuffle=False, num_workers=0)
print('num_train_files: '+str(len(train_dataset.filepaths)))
print('num_val_files: '+str(len(val_dataset.filepaths)))

num_train_files: 287700
num_val_files: 37404
num_train_files: 287700
num_val_files: 37404


In [14]:

V

12

In [17]:
meta_num = 200
meta_val_dataset = MultiviewImgDataset(val_path, scale_aug=False, rot_aug=False, num_views=num_views)
meta_val_dataset.val_set_select(meta_num)
V,C,H,W = meta_val_dataset[0][1].size()
meta_data_x = torch.Tensor(meta_num, V,C,H,W)
meta_data_y = torch.Tensor(meta_num)
for i, m in enumerate(meta_val_dataset):
    meta_data_y[i] = m[0]
    meta_data_x[i] = m[1]
meta_data_y = meta_data_y.int()
meta_data_x

tensor([[[[[0.7604, 0.7604, 0.7604,  ..., 0.7604, 0.7604, 0.7604],
           [0.7604, 0.7604, 0.7604,  ..., 0.7604, 0.7604, 0.7604],
           [0.7604, 0.7604, 0.7604,  ..., 0.7604, 0.7604, 0.7604],
           ...,
           [0.7604, 0.7604, 0.7604,  ..., 0.7604, 0.7604, 0.7604],
           [0.7604, 0.7604, 0.7604,  ..., 0.7604, 0.7604, 0.7604],
           [0.7604, 0.7604, 0.7604,  ..., 0.7604, 0.7604, 0.7604]],

          [[0.7604, 0.7604, 0.7604,  ..., 0.7604, 0.7604, 0.7604],
           [0.7604, 0.7604, 0.7604,  ..., 0.7604, 0.7604, 0.7604],
           [0.7604, 0.7604, 0.7604,  ..., 0.7604, 0.7604, 0.7604],
           ...,
           [0.7604, 0.7604, 0.7604,  ..., 0.7604, 0.7604, 0.7604],
           [0.7604, 0.7604, 0.7604,  ..., 0.7604, 0.7604, 0.7604],
           [0.7604, 0.7604, 0.7604,  ..., 0.7604, 0.7604, 0.7604]],

          [[0.7604, 0.7604, 0.7604,  ..., 0.7604, 0.7604, 0.7604],
           [0.7604, 0.7604, 0.7604,  ..., 0.7604, 0.7604, 0.7604],
           [0.7604, 0.7604

In [19]:
meta_data_y[0]

tensor(13, dtype=torch.int32)

In [8]:

print(meta_val_loader[0])

TypeError: 'DataLoader' object is not subscriptable

In [20]:
# next(iter(meta_val_loader))

# data = enumerate(meta_val_loader)
list(meta_val_loader)[0]

3

In [4]:
# STAGE 1
log_dir = name+'_stage_1'
create_folder(log_dir)
cnet = SVCNN(name, nclasses=55, pretraining=pretraining, cnn_name=cnn_name)

optimizer = optim.Adam(cnet.parameters(), lr=lr, weight_decay=weight_decay)

n_models_train = num_models*num_views

train_dataset = SingleImgDataset(train_path, scale_aug=False, rot_aug=False, num_models=n_models_train, num_views=num_views)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=svcnn_bs, shuffle=True, num_workers=0)

val_dataset = SingleImgDataset(val_path, scale_aug=False, rot_aug=False, test_mode=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=svcnn_bs, shuffle=False, num_workers=0)
print('num_train_files: '+str(len(train_dataset.filepaths)))
print('num_val_files: '+str(len(val_dataset.filepaths)))

num_train_files: 279972
num_val_files: 37404


In [ ]:
trainer = ModelNetTrainer(cnet, train_loader, val_loader, optimizer, nn.CrossEntropyLoss(), 'svcnn', log_dir, num_views=1)
trainer.train(30)

epoch 1, step 1: train_loss 4.163; train_acc 0.031
epoch 1, step 2: train_loss 4.015; train_acc 0.031
epoch 1, step 3: train_loss 3.995; train_acc 0.031
epoch 1, step 4: train_loss 4.020; train_acc 0.000
epoch 1, step 5: train_loss 4.009; train_acc 0.031
epoch 1, step 6: train_loss 4.041; train_acc 0.000
epoch 1, step 7: train_loss 4.140; train_acc 0.000
epoch 1, step 8: train_loss 3.967; train_acc 0.031
epoch 1, step 9: train_loss 3.957; train_acc 0.062
epoch 1, step 10: train_loss 3.966; train_acc 0.000
epoch 1, step 11: train_loss 3.864; train_acc 0.000
epoch 1, step 12: train_loss 3.981; train_acc 0.062
epoch 1, step 13: train_loss 3.859; train_acc 0.062
epoch 1, step 14: train_loss 3.856; train_acc 0.125
epoch 1, step 15: train_loss 3.891; train_acc 0.125
epoch 1, step 16: train_loss 3.972; train_acc 0.031
epoch 1, step 17: train_loss 3.850; train_acc 0.031
epoch 1, step 18: train_loss 3.906; train_acc 0.094
epoch 1, step 19: train_loss 3.857; train_acc 0.062
epoch 1, step 20: tra

epoch 1, step 158: train_loss 3.327; train_acc 0.125
epoch 1, step 159: train_loss 3.403; train_acc 0.188
epoch 1, step 160: train_loss 3.385; train_acc 0.125
epoch 1, step 161: train_loss 3.797; train_acc 0.125
epoch 1, step 162: train_loss 3.587; train_acc 0.156
epoch 1, step 163: train_loss 3.414; train_acc 0.219
epoch 1, step 164: train_loss 3.404; train_acc 0.188
epoch 1, step 165: train_loss 3.242; train_acc 0.156
epoch 1, step 166: train_loss 3.253; train_acc 0.219
epoch 1, step 167: train_loss 3.079; train_acc 0.219
epoch 1, step 168: train_loss 3.224; train_acc 0.188
epoch 1, step 169: train_loss 3.438; train_acc 0.188
epoch 1, step 170: train_loss 3.761; train_acc 0.156
epoch 1, step 171: train_loss 3.366; train_acc 0.188
epoch 1, step 172: train_loss 3.369; train_acc 0.156
epoch 1, step 173: train_loss 3.309; train_acc 0.156
epoch 1, step 174: train_loss 3.384; train_acc 0.188
epoch 1, step 175: train_loss 3.310; train_acc 0.156
epoch 1, step 176: train_loss 3.220; train_acc

epoch 1, step 313: train_loss 3.276; train_acc 0.281
epoch 1, step 314: train_loss 3.573; train_acc 0.094
epoch 1, step 315: train_loss 2.807; train_acc 0.312
epoch 1, step 316: train_loss 3.582; train_acc 0.125
epoch 1, step 317: train_loss 3.282; train_acc 0.250
epoch 1, step 318: train_loss 3.237; train_acc 0.219
epoch 1, step 319: train_loss 2.993; train_acc 0.281
epoch 1, step 320: train_loss 3.306; train_acc 0.156
epoch 1, step 321: train_loss 2.941; train_acc 0.250
epoch 1, step 322: train_loss 3.266; train_acc 0.219
epoch 1, step 323: train_loss 3.256; train_acc 0.156
epoch 1, step 324: train_loss 2.795; train_acc 0.250
epoch 1, step 325: train_loss 3.216; train_acc 0.219
epoch 1, step 326: train_loss 2.682; train_acc 0.312
epoch 1, step 327: train_loss 3.571; train_acc 0.156
epoch 1, step 328: train_loss 3.120; train_acc 0.219
epoch 1, step 329: train_loss 2.749; train_acc 0.375
epoch 1, step 330: train_loss 3.626; train_acc 0.156
epoch 1, step 331: train_loss 2.817; train_acc

epoch 1, step 468: train_loss 3.036; train_acc 0.312
epoch 1, step 469: train_loss 3.177; train_acc 0.188
epoch 1, step 470: train_loss 2.869; train_acc 0.188
epoch 1, step 471: train_loss 2.943; train_acc 0.250
epoch 1, step 472: train_loss 3.058; train_acc 0.250
epoch 1, step 473: train_loss 2.820; train_acc 0.219
epoch 1, step 474: train_loss 3.490; train_acc 0.219
epoch 1, step 475: train_loss 3.096; train_acc 0.219
epoch 1, step 476: train_loss 2.876; train_acc 0.344
epoch 1, step 477: train_loss 3.009; train_acc 0.188
epoch 1, step 478: train_loss 2.901; train_acc 0.250
epoch 1, step 479: train_loss 3.547; train_acc 0.125
epoch 1, step 480: train_loss 3.001; train_acc 0.250
epoch 1, step 481: train_loss 2.914; train_acc 0.281
epoch 1, step 482: train_loss 3.259; train_acc 0.250
epoch 1, step 483: train_loss 3.200; train_acc 0.281
epoch 1, step 484: train_loss 3.327; train_acc 0.281
epoch 1, step 485: train_loss 3.055; train_acc 0.281
epoch 1, step 486: train_loss 2.544; train_acc

epoch 1, step 623: train_loss 2.874; train_acc 0.281
epoch 1, step 624: train_loss 2.669; train_acc 0.375
epoch 1, step 625: train_loss 3.011; train_acc 0.219
epoch 1, step 626: train_loss 2.801; train_acc 0.219
epoch 1, step 627: train_loss 2.377; train_acc 0.438
epoch 1, step 628: train_loss 2.826; train_acc 0.281
epoch 1, step 629: train_loss 2.364; train_acc 0.406
epoch 1, step 630: train_loss 2.642; train_acc 0.375
epoch 1, step 631: train_loss 3.147; train_acc 0.062
epoch 1, step 632: train_loss 2.813; train_acc 0.375
epoch 1, step 633: train_loss 2.953; train_acc 0.250
epoch 1, step 634: train_loss 3.187; train_acc 0.312
epoch 1, step 635: train_loss 2.356; train_acc 0.375
epoch 1, step 636: train_loss 3.248; train_acc 0.250
epoch 1, step 637: train_loss 2.477; train_acc 0.438
epoch 1, step 638: train_loss 2.582; train_acc 0.344
epoch 1, step 639: train_loss 2.629; train_acc 0.375
epoch 1, step 640: train_loss 2.599; train_acc 0.312
epoch 1, step 641: train_loss 3.530; train_acc

epoch 1, step 778: train_loss 3.537; train_acc 0.094
epoch 1, step 779: train_loss 2.556; train_acc 0.344
epoch 1, step 780: train_loss 2.702; train_acc 0.219
epoch 1, step 781: train_loss 2.663; train_acc 0.375
epoch 1, step 782: train_loss 2.627; train_acc 0.281
epoch 1, step 783: train_loss 2.990; train_acc 0.219
epoch 1, step 784: train_loss 2.596; train_acc 0.281
epoch 1, step 785: train_loss 2.905; train_acc 0.250
epoch 1, step 786: train_loss 3.225; train_acc 0.156
epoch 1, step 787: train_loss 2.568; train_acc 0.250
epoch 1, step 788: train_loss 2.824; train_acc 0.250
epoch 1, step 789: train_loss 2.904; train_acc 0.281
epoch 1, step 790: train_loss 2.560; train_acc 0.344
epoch 1, step 791: train_loss 2.907; train_acc 0.156
epoch 1, step 792: train_loss 2.686; train_acc 0.344
epoch 1, step 793: train_loss 2.917; train_acc 0.156
epoch 1, step 794: train_loss 2.917; train_acc 0.281
epoch 1, step 795: train_loss 2.884; train_acc 0.406
epoch 1, step 796: train_loss 3.016; train_acc

In [4]:
# STAGE 2
log_dir = name+'_stage_2'
create_folder(log_dir)
cnet_2 = MVCNN(name, cnet, nclasses=40, cnn_name=cnn_name, num_views=num_views)
del cnet

# Adam Optimizer是对SGD的扩展，可以代替经典的随机梯度下降法来更有效地更新网络权重。
optimizer = optim.Adam(cnet_2.parameters(), lr=lr, weight_decay=weight_decay, betas=(0.9, 0.999))

train_dataset = MultiviewImgDataset(train_path, scale_aug=False, rot_aug=False, num_models=n_models_train, num_views=num_views)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batchSize, shuffle=False, num_workers=0)# shuffle needs to be false! it's done within the trainer

val_dataset = MultiviewImgDataset(val_path, scale_aug=False, rot_aug=False, num_views=num_views)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batchSize, shuffle=False, num_workers=0)
print('num_train_files: '+str(len(train_dataset.filepaths)))
print('num_val_files: '+str(len(val_dataset.filepaths)))
trainer = ModelNetTrainer(cnet_2, train_loader, val_loader, optimizer, nn.CrossEntropyLoss(), 'mvcnn', log_dir, num_views=num_views)
trainer.train(30)


NameError: name 'create_folder' is not defined

In [5]:
folders = '''


'''
fs = folders.split(" ")